# Защита персональных данных клиентов

**Условия задачи**

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.


**Описание данных**

Набор данных находится в файле /datasets/insurance.csv. 

- **Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.
- **Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

**План выполнения работы**

[1  Загрузка данных](#section1)

[2  Умножение матриц](#section2)

[3  Алгоритм преобразования](#section3)

[4  Проверка алгоритма](#section4)

- [4.1  Определение признаков и целевого признака](#section4.1)

- [4.2  Обучение модели без преобразование и получение метрики R2](#section4.2)

- [4.3  Создание рандомной обратимой матрицы](#section4.3)

- [4.5  Умножение обратимой матрицы на матрицу признаков](#section4.4)

- [4.6  Обучение преобразованной модели и получение метрики R2](#section4.5)

- [4.7  Сравнение метрик R2 полученных на преобразованной модели и модели начальной](#section4.6)

- [4.8  Для проверки расшифруем полученную матрицу и сравним результат с начальной](#section4.7)

[5  Общий вывод](#section5)

<a id='section1'></a>
## Загрузка данных

In [1]:
# Загружаем необходимы библиотеки для работы
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
# сохраним датафрейм в переменную
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
# посмотрим общий вид таблицы
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
# смотрим на общую информацию по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
# проверим наличие дубликатов
df.duplicated().sum()

153

Дубликаты имеются, однако вариативность данных достаточно низкая, и вполне возможно получить данные пользователей с совпадениями. По условию задачи требуется защитить данные и в данном случае оставим дубликаты без изменений.

In [6]:
# посмотрим наличие пропусков
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [7]:
# посмотрим на распределение данных, есть ли какие-то вбросы в данных, значения которые не укладываются в "нормальные"
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Вывод:**
- Данные предоставлены в предобработанном виде и готовы для дальнейшего исследования
- Аномалии в данных не наблюдаются

<a id='section2'></a>
## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**
При умножение признаки на обратимую матрицу, качество линейной регресии не изменится

**Обоснование:**

Формула Squared Error:
$$
SE = \sum_ {i = 1} ^ {n} (y_i + a_i)^2
$$

Формула предсказания в матричной виде:

$$
a = Xw = X(X^T X)^{-1}X^T y
$$

Пусть матрица Z является обратимой:
Умножаем матрицу X на обратимую Z в формуле предсказания:

$$
a^{'} = (XZ)((XZ)^TXZ)^{-1}(XZ)^Ty
$$


Преобразуем формулу с помошью свойства транспортирования матрицы:

$$
(AB)^T = B^TA^T
$$
$$
a^{'} = (XZ)(Z^TX^TXZ)^{-1}Z^TX^Ty
$$

Преобразуем формулу с помошью следующего свойства свойства:

$$
(ABС)^{-1} = С^{-1}B^{-1}A^{-1}
$$
$$
a^{'} = (XZ)Z^{-1}(X^TX)^{-1}(Z^T)^{-1}Z^TX^Ty
$$

Применяем тождетсва:

$$
AA^{-1} = A^{-1}A = E  
$$

Где:

$
E 
$ — единичная матрица

$$
a^{'} = XE(X^TX)^{-1}EX^Ty
$$

Применяем тождетсва:

$$
AE = EA = A
$$
$$
a^{'} = X(X^TX)^{-1}X^Ty
$$

Получаем что:

$$
a = a^{'}
$$

Соответсвено :

$$
\sum_ {i = 1} ^ {n} (y_i + a_i)^2 = \sum_ {i = 1} ^ {n} (y_i + a_i^{'})^2
$$
 
Что является доказательством предположения того что при умножения признаки на обратимую матрицу, качество линейной регресии не изменится.

<a id='section3'></a>
## Алгоритм преобразования

**Алгоритм**
- Определение признаков и целевого признака
- Обучение модели без преобразование и получение метрики R2
- Создание рандомной обратимой матрицы
- Умножение обратимой матрицы на матрицу признаков
- Обучение преобразованной модели и получение метрики R2
- Сравнение метрик R2 полученных на преобразованной модели и модели начальной 
- Для проверки расшифруем полученную матрицу и сравним результат с начальной
- Вывод

**Обоснование**

Алгоритм основан на доказательстве в предыдущем пункте

<a id='section4'></a>
## Проверка алгоритма

<a id='section4.1'></a>
### Определение признаков и целевого признака

In [8]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

<a id='section4.2'></a>
### Обучение модели без преобразование и получение метрики R2

In [9]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv((X.T @ X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        print(w)
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [10]:
model_zero = LinearRegression()
model_zero.fit(features, target)
predictions = model_zero.predict(features)
R2_zero = r2_score(target, predictions)

[-9.38235504e-01  7.92580543e-03  3.57083050e-02 -1.70080492e-07
 -1.35676623e-02]


<a id='section4.3'></a>
### Создание рандомной обратимой матрицы

In [11]:
# Создание рандомной матрицы
random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))

Создание обратимой матрицы через np.linalg.inv() и проверка через умножение рандомной и обратимой, должна получится единичная матрица

In [12]:
random_matrix @ np.linalg.inv(random_matrix)

array([[ 1.00000000e+00, -1.98157100e-17, -3.95275185e-17,
         1.15349142e-16],
       [-1.05854737e-16,  1.00000000e+00, -3.34863499e-17,
        -1.49376218e-17],
       [ 1.43664884e-17,  2.90122843e-17,  1.00000000e+00,
        -2.01350736e-18],
       [ 2.65085043e-16,  1.18794076e-16, -1.70602328e-17,
         1.00000000e+00]])

<a id='section4.4'></a>
### Умножение обратимой матрицы на матрицу признаков

In [13]:
secret_features = features.dot(random_matrix)

<a id='section4.5'></a>
### Обучение преобразованной модели и получение метрики R2

In [14]:
model_secret = LinearRegression()
model_secret.fit(secret_features, target)
predictions = model_secret.predict(secret_features)
R2_secret = r2_score(target, predictions)

[-0.93823553  0.00196286 -0.01198092  0.01664193  0.00887001]


<a id='section4.6'></a>
### Сравнение метрик R2 полученных на преобразованной модели и модели начальной 

In [15]:
df_r2_result = pd.DataFrame({'Тип данных' : 'нешифрованные данные',
                                    'R2-score'   : R2_zero}, index=[0])
df_r2_result = df_r2_result.append({'Тип данных' : 'шифрованные данные',
                                                  'R2-score'   : R2_secret}, ignore_index = True).reset_index(drop = True)
df_r2_result

,Тип данных,R2-score
0,нешифрованные данные,0.424946
1,шифрованные данные,0.424946


<a id='section4.7'></a>
### Для проверки расшифруем полученную матрицу и сравним результат с начальной

In [16]:
def decode(secret_features):
    decoded_features = round(abs(secret_features.dot(np.linalg.inv(random_matrix))))
    for i in decoded_features.iloc[:, [0, 3]]:
        decoded_features[i] = decoded_features[i].astype(int)
    decoded_features.columns = features.columns
    return decoded_features

In [17]:
decode(secret_features)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [18]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


<a id='section5'></a>
## Общий вывод

- Умножив признаки на обратимую матрицу, качество линейной регрессии не изменилось

- Предложенный алгоритм преобразования данных справляется для решения задачи шифрования признаков

- Зашифрованные данные так же пригодны для предсказаний с помощью линейной регресии

- Алгоритм позволяет дешифровать данные при необходимости

- Случайность реализованная в алгоритме каждый раз меняется при запуске, что делает зашифрованные данные каждый раз новыми. Такой вид исходных данных сложно востановить, не имея исходных данных

- Реализованный алгоритм полностью удовлетворяет поставленной задаче: "Защиты персональных данных клиентов страховой компании"